In [24]:
import deepchem as dc
import os
import sys
os.environ['DGLBACKEND'] = 'pytorch'
import numpy as np
import pandas as pd
import dgl
import torch
from deepchem.feat.graph_data import GraphData
from deepchem.models import MPNNModel
from rdkit import Chem
from rdkit.Chem import Descriptors
import math

import copy
sys.path.append('../')
from utils import *
from save_data import *
from run_experiments import run_experiments
import matplotlib.pyplot as plt
from itertools import product
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

In [3]:
df = pd.read_csv('../expt_phys_data/full_dataset_with_scaffold_split.csv').fillna('')
df

,id,smiles,iupac name,scaffold,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split,scaffold_split
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,,-2.49,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,2,1,0
1,mobley_1019269,CCCCO,butan-1-ol,,-4.72,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,2,0,1
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",c1ccc(-c2ccccc2)cc1,-3.04,-1.083,-2.344750,-2.113720,-0.459959,0.899,-1.348,0,False,2,0,1
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,C1CCCCC1,-4.59,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,1,0,2
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,c1ccccc1,-3.82,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",,-0.78,1.024,-0.252349,-0.595880,-0.520564,-0.090,-1.126,0,True,1,0,0
638,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,c1ccccc1,-0.95,-0.575,-1.201310,-0.421314,-0.758853,0.059,-1.641,0,True,0,0,0
639,mobley_994483,CCBr,bromoethane,,-0.74,0.487,-0.821768,-0.062573,-1.089090,-0.095,-0.818,0,True,0,0,0
640,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,,-1.21,0.140,-1.111810,0.499588,-0.765837,0.032,-1.358,0,True,0,0,0


In [7]:
set(df.smiles.apply(list).sum())

{'#',
 '(',
 ')',
 '+',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '=',
 '@',
 'B',
 'C',
 'F',
 'H',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[',
 '\\',
 ']',
 'c',
 'l',
 'n',
 'r',
 's'}

In [14]:
subset = df.copy()
chars = ['l','S','s','B','F','I','P','r']
for c in chars:
    subset = subset[~ subset.smiles.str.contains(c)]
subset

,id,smiles,iupac name,scaffold,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split,scaffold_split
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,,-2.49,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,2,1,0
1,mobley_1019269,CCCCO,butan-1-ol,,-4.72,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,2,0,1
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,C1CCCCC1,-4.59,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,1,0,2
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,c1ccccc1,-3.82,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0,0,0
5,mobley_1075836,CCC(=O)OC,methyl propanoate,,-2.93,-3.652,-4.291680,-3.664740,-4.614940,-4.915,-5.664,0,True,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,mobley_9794857,CC(C)(C)OC,2-methoxy-2-methyl-propane,,-2.21,-0.691,-0.971671,-1.208270,-1.423510,-0.560,-1.286,0,True,1,0,1
634,mobley_9838013,CCC(C)(C)O,2-methylbutan-2-ol,,-4.43,-2.933,-2.578670,-2.477790,-3.790770,-2.224,-3.505,0,True,1,0,1
635,mobley_9883303,CCC[C@H](C)CC,3-methylhexane,,2.71,2.810,2.343420,2.291730,1.805270,3.052,1.502,0,True,0,0,1
636,mobley_9897248,CC(=CCC/C(=C\CO)/C)C,"(2Z)-3,7-dimethylocta-2,6-dien-1-ol",,-4.78,-2.597,-3.518640,-3.437290,-5.478800,-3.230,-4.754,0,True,0,0,0


In [26]:
subset2 = df[~df.id.isin(subset.id)]
pd.set_option('display.max_rows', 10)
# subset2

In [22]:
temp = subset
split = 'stratified_split_8'
dataset_names = ['train','val','test']
phys_models = ['tip3p','cha','zap9','mbondi','asc','igb5','null']
for i in range(4):
    
    t = temp[temp[split] == i]
    if(i == 3):
        t = temp
        print('full',len(t))
    else:
        print(dataset_names[i],len(t))
    for p in phys_models:
        print(p,rmsd(t.expt,t[p]))

train 328
tip3p 1.3487857400495364
cha 1.575136690821102
zap9 1.4114031698213896
mbondi 1.9272436172099885
asc 1.9924677278042582
igb5 2.36943855652995
null 5.551989123882297
val 49
tip3p 1.4996886683711443
cha 2.093845006254902
zap9 1.8455401219174516
mbondi 2.329327903482902
asc 2.3923877108829315
igb5 2.7118802085096094
null 5.682175136839263
test 54
tip3p 1.2266682216173719
cha 1.3303493958251746
zap9 1.554757139106725
mbondi 2.2539685750580203
asc 2.244897733379704
igb5 2.736222267341653
null 5.064934279855886
full 431
tip3p 1.3522894396312806
cha 1.6150267303959436
zap9 1.4852896532668551
mbondi 2.0199025732502167
asc 2.074441334236265
igb5 2.4590021644991698
null 5.508446182940161


In [44]:
temp = df
split = 'stratified_split_8'
dataset_names = ['train','val','test']
phys_models = ['tip3p','cha','zap9','mbondi','asc','igb5','null']
for i in range(4):
    
    t = temp[temp[split] == i]
    if(i == 3):
        t = temp
        print('full',len(t))
    else:
        print(dataset_names[i],len(t))
    for p in phys_models:
        print(p,rmsd(t.expt,t[p]))

train 482
tip3p 1.439803209524717
cha 1.6285670323718344
zap9 1.5703079552136028
mbondi 2.137337931152265
asc 2.3102512751553337
igb5 2.663562315686188
null 5.446609322508196
val 80
tip3p 2.026509304814562
cha 2.3718983358857706
zap9 2.2325395874643017
mbondi 2.787031100279458
asc 3.4624858393212237
igb5 3.6372014825962005
null 5.329005653027589
test 80
tip3p 1.5725272255194822
cha 1.4634683233022971
zap9 1.580197705793768
mbondi 2.333009506930915
asc 2.5314665092590105
igb5 2.9949296298076855
null 5.249951666444177
full 642
tip3p 1.5415171295667343
cha 1.7202235052613517
zap9 1.6684018476881877
mbondi 2.2529720527573227
asc 2.5098963780722974
igb5 2.844759897669645
null 5.4079121825760845


In [27]:
df

,id,smiles,iupac name,scaffold,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split,scaffold_split
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,,-2.49,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,2,1,0
1,mobley_1019269,CCCCO,butan-1-ol,,-4.72,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,2,0,1
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",c1ccc(-c2ccccc2)cc1,-3.04,-1.083,-2.344750,-2.113720,-0.459959,0.899,-1.348,0,False,2,0,1
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,C1CCCCC1,-4.59,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,1,0,2
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,c1ccccc1,-3.82,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",,-0.78,1.024,-0.252349,-0.595880,-0.520564,-0.090,-1.126,0,True,1,0,0
638,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,c1ccccc1,-0.95,-0.575,-1.201310,-0.421314,-0.758853,0.059,-1.641,0,True,0,0,0
639,mobley_994483,CCBr,bromoethane,,-0.74,0.487,-0.821768,-0.062573,-1.089090,-0.095,-0.818,0,True,0,0,0
640,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,,-1.21,0.140,-1.111810,0.499588,-0.765837,0.032,-1.358,0,True,0,0,0


In [42]:
np.random.seed(10)
expt = df.expt.to_numpy()
train_index = list(psuedoScramble(expt, bins=int(len(expt)/8)))

test_index = []
val_index = []
binw = len(expt)/float(int(len(expt)/8))
print(binw)
i=0
for j in range(len(train_index)//8):
    k = j*int(binw-1)+2
    test_index.append(train_index.pop(k))
for i in range(len(train_index)//7):
    k = i*int(binw-2)+2
    val_index.append(train_index.pop(k))

8.025


In [31]:
argsort = np.argsort(expt)
argsort[-5:]

array([443, 436, 270, 103,  45])

In [32]:
train_index = list(psuedoScramble(expt, bins=int(len(expt)/10)))
train_index[-5:]

[443, 348, 8, 45, 436]

In [43]:
df['stratified_split_8'] = 0
df.loc[df.index[val_index],'stratified_split_8'] = 1
df.loc[df.index[test_index],'stratified_split_8'] = 2